In [ ]:
from surgeon_recording.reader import Reader
from glob import glob
from os.path import join
import os
import numpy as np
import itertools
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def data_average(df):
    return df.mean()

In [ ]:
def quaternion_average(df):
    # function taken from
    # https://stackoverflow.com/questions/12374087/average-of-multiple-quaternions
    A = df.transpose().dot(df)
    w, v = np.linalg.eig(A)
    q = v[:, w.argmax()].real
    q = -q if q[0] < 0 else q
    return q

In [ ]:
def downsample(data, time_vector, average_function, selected_columns):
    def insert_row(data, row, labels=None):
        return data.append(pd.Series(row, labels), ignore_index=True)
    
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=selected_columns)
    
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]
        start_time = current_time_index
        
        while current_time_index < data.shape[0] and data['relative_time'].iloc[current_time_index] < t:
            current_time_index = current_time_index + 1
        stop_time = current_time_index
        
        average_data = average_function(data.iloc[start_time:stop_time][selected_columns]) if stop_time != start_time else np.zeros(len(selected_columns))
        downsampled_data = insert_row(downsampled_data, average_data, selected_columns)
    return downsampled_data

# Data choice

In [ ]:
experiment = 'december'

In [ ]:
fruits = ['orange']

In [ ]:
cut_qualities = ['good'] # good / shallow / deep

In [ ]:
frames = ['ExactoKnife']

In [ ]:
force_components = ['force'] # force and/or torque

# Data extraction

In [ ]:
reader = Reader()

In [ ]:
data_folder = join('..', 'data', experiment)

In [ ]:
folders  = [join(data_folder, f, cq) for f in fruits for cq in cut_qualities]

In [ ]:
runs = list(itertools.chain.from_iterable([x[0] for x in os.walk(f)][1:] for f in folders))

In [ ]:
opt_position_header = list(itertools.chain.from_iterable((f + '_x', f + '_y', f + '_z') for f in frames))
opt_orient_header = list(itertools.chain.from_iterable((f + '_qx', f + '_qy', f + '_qz', f + '_qw') for f in frames))

In [ ]:
ft_desired_header = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in force_components))

In [ ]:
timeseries = []

for r in runs:
    reader.play(r)
    timestamp = reader.data['ft_sensor']['relative_time']
    # exctract force data
    force_data = reader.data['ft_sensor'][ft_desired_header].reset_index(drop=True)
    # downsample optitrack data
    opt_position_data = downsample(reader.data['optitrack'], timestamp, data_average, opt_position_header)
    opt_orient_data = downsample(reader.data['optitrack'], timestamp, quaternion_average, opt_orient_header)
    # merge the data
    merge_data = pd.concat([opt_position_data, opt_orient_data, force_data], axis=1)
    # store in the list
    timeseries.append(merge_data)

# Transformation

In [ ]:
O_T_FT = np.array([[-0.0032, 1.0000, 0.0059, 0.0481],
                   [-1.0000, -0.0031, -0.0085, 0.9708],
                   [-0.0085, -0.0059, 0.9999, -0.3783],
                   [0, 0, 0, 1.0000]])

In [ ]:
transform_header =  ['opt_force_' + axis for axis in ['x', 'y', 'z']]

In [ ]:
for i, ts in enumerate(timeseries):
    transform =  O_T_FT[:3, :3].dot(ts[ft_desired_header].T) + np.tile(O_T_FT[:3, 3], (len(ts),1)).T
    timeseries[i] = pd.concat([ts, pd.DataFrame(data=transform.T, columns=transform_header)], axis=1, sort=False)